In [15]:
# import data (cleaned & embedded data from partener Duanyue Yun)
import pandas as pd
df = pd.read_csv('~/Desktop/NYC_airbnb.csv')
df.drop(df.columns[0],axis=1,inplace=True)
df.head(10)

,listing_id,price,bathrooms,bedrooms,is_superhost,lat,lng,listing_name,person_capacity,room_type_category,...,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14
0,27678634,20,1.0,1.0,False,40.668720,-73.953430,Brooklyn Chill & Explore,2,private_room,...,-0.114761,-0.488204,0.567969,-0.038650,-0.655065,-0.280678,-0.556897,0.289369,0.020916,-0.163320
1,2062476,23,1.5,1.0,False,40.709460,-74.074320,A room in private house - 20-35 minutes from NYC,2,private_room,...,0.033249,-0.163629,0.512907,-0.508940,0.011784,-0.034007,-0.020552,0.165408,0.026181,0.367658
2,17157306,29,1.0,1.0,False,40.738020,-74.056990,Modern Room 20 Minutes to Times Square,2,private_room,...,1.061535,0.263956,-0.623987,-0.173793,0.013040,0.139797,-0.132379,0.008352,-0.564884,0.176254
3,401039,27,1.0,1.0,False,40.848860,-73.912230,Big and Comfy room near Yankee1,4,private_room,...,0.060883,-0.406233,0.136488,0.053615,-0.250396,-0.299606,-0.059454,-0.392709,0.115975,-0.065035
4,12798739,29,2.0,1.0,False,40.678509,-74.200249,Good Deal! Cozy room Near NYC/EWR,2,private_room,...,-0.126636,-0.517378,0.251064,-0.163608,0.063405,-0.204988,0.072095,0.161953,0.182331,0.278517
5,21282152,33,2.0,1.0,False,40.700310,-73.908280,Pop House 4min walk to L train 12mins to Manha...,4,private_room,...,0.294604,-0.059078,0.377726,0.204856,-0.004628,-0.013008,-0.175447,-0.471408,-0.147531,0.051353
6,858695,30,1.0,1.0,False,40.840580,-73.913820,Very Large Private Room on quiet st,3,private_room,...,-0.460543,0.023108,0.227358,-0.161204,-0.079583,0.204344,-0.140348,0.245019,-0.064191,-0.041311
7,20194665,39,1.0,1.0,False,40.780880,-73.908190,"20min To Times Square, 10min LGA",2,private_room,...,0.945571,0.268598,-0.989545,-0.181290,0.109903,0.389990,-0.069701,0.424437,-0.769621,-0.115390
8,28227966,32,1.0,1.0,True,40.659250,-73.893960,Beautiful One Bed,3,private_room,...,0.833106,0.354080,0.107509,0.320108,0.051589,0.590938,-0.317367,-0.421662,0.310910,-0.242722
9,23669328,32,1.0,1.0,False,40.687540,-73.914110,Private Bedroom in Apartment in Bushwick & Bed...,2,private_room,...,-0.342958,-0.404081,0.431380,0.042713,0.293898,0.279978,-0.143272,0.467333,0.155734,-0.294396


In [31]:
features2.isnull().sum()

price               0
bathrooms           0
bedrooms            0
is_superhost        0
lat                 0
lng                 0
person_capacity     0
can_instant_book    0
min_nights          0
max_nights          0
entire_home         0
hotel_room          0
private_room        0
shared_room         0
dtype: int64

## generate feature X and label y
### features: with listing name
### features2: without listing name

In [16]:
cols = ["listing_id", "listing_name", "cleaned_name", "reviews_per_month"]
features = df.drop(columns=cols)

In [17]:
features = features.join(pd.get_dummies(features.room_type_category))

In [18]:
cols = ["room_type_category"]
features = features.drop(columns=cols)

In [30]:
cols = ["c_0","c_1","c_2","c_3","c_4","c_5","c_6","c_7","c_8","c_9","c_10","c_11","c_12","c_13","c_14"]
features2 = features.drop(columns=cols)

In [21]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [32]:
y = np.array(df['reviews_per_month'])

x = np.array(features)
x2 = np.array(features2)

In [33]:
print(np.shape(x), np.shape(x2), np.shape(y))

(3337, 29) (3337, 14) (3337,)


## generate train & test data

In [34]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)
x2_train, x2_test, y_train, y_test = train_test_split(x2, y, test_size=0.2, random_state=123)


## try XGBoost model

In [40]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(x_train,y_train)

preds = xg_reg.predict(x_test)

# Calculate the mean_sqaured_error
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
# Calculate the absolute errors
errors = abs(preds - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'reviews.')

[12:31:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 4.412340
Mean Absolute Error: 2.41 reviews.


## Grid Search

In [26]:
from sklearn.model_selection import GridSearchCV
parameters = {
              'max_depth': [7, 8, 9, 10],
              'eta': [0.05, 0.1, 0.25],
              'gamma': [0, 0.05, 0.15],
              'min_child_weight': [0, 1, 2, 3],
              'max_delta_step': [0, 2, 4], 
              'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15]
    

}

xlf = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.68, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)


# 有了gridsearch我们便不需要fit函数
gsearch = GridSearchCV(xlf, param_grid=parameters, scoring='neg_mean_squared_error', cv=3)
gsearch.fit(x_train, y_train)

print("Best score: %0.3f" % gsearch.best_score_)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

[18:44:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

## best parameter 

In [27]:
print("Best score: %0.3f" % abs(gsearch.best_score_)**0.5)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 4.199
Best parameters set:
	eta: 0.05
	gamma: 0.15
	learning_rate: 0.15
	max_delta_step: 0
	max_depth: 7
	min_child_weight: 0


In [28]:
cv_result = pd.DataFrame.from_dict(gsearch.cv_results_)
with open('cv_result.csv','w') as f:
    cv_result.to_csv(f)

In [38]:
xg_reg2 = xgb.XGBRegressor(objective ='reg:linear', eta = 0.05, gamma = 0.15, learning_rate = 0.15, max_delta_step = 0, max_depth = 7, min_child_weight = 0)          

xg_reg2.fit(x2_train,y_train)

[22:09:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.05, gamma=0.15,
             importance_type='gain', learning_rate=0.15, max_delta_step=0,
             max_depth=7, min_child_weight=0, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [39]:
preds = xg_reg2.predict(x2_test)

# Calculate the mean_sqaured_error
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

# Calculate the absolute errors
errors = abs(preds - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'reviews.')

RMSE: 4.275197
Mean Absolute Error: 2.31 reviews.
